### Alice in SEQUENCELAND 

##### Generating text based on Alice in wonderland with LSTM arquitecture 

In [1]:
import nltk
from nltk.corpus import gutenberg
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
alice = gutenberg.raw(fileids='carroll-alice.txt')

In [4]:
len(alice)

144395

In [5]:
alice[0:100]

"[Alice's Adventures in Wonderland by Lewis Carroll 1865]\n\nCHAPTER I. Down the Rabbit-Hole\n\nAlice was"

In [6]:
chars = sorted(list(set(alice)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 73


In [7]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 30 # gives us an idea of the seed for the generation 
step = 2
sentences = []
next_chars = []


In [8]:
for i in range(0, len(alice) - maxlen, step):
    sentences.append(alice[i: i + maxlen])
    next_chars.append(alice[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 72183


In [9]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [10]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(alice) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0]: #The higher diversity the randomness in the text 
        print('----- diversity:', diversity)

        generated = ''
        sentence = alice[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=32,
          epochs=160,
          callbacks=[print_callback])

Epoch 1/160
72160/72183 [============================>.] - ETA: 0s - loss: 2.0682
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "e found it made no mark; but h"
e found it made no mark; but her went of the sell and and her went of the seally and her sell and said the go the she said the should see thing and her went of the seal me the seal she said the seat she went of the sent of the seat and the say and thing, 'in the was the say and and her sound the seat the sent of the should she went her was and her said the seat to sell and the was she went of the seally and she went of thing w
----- diversity: 0.5
----- Generating with seed: "e found it made no mark; but h"
e found it made no mark; but her a tremed went to she thing in the did you here fin she dear.

'The could be a house me never your her the sent her was of the did as the here the in that she wrill go, and and the sent, and wait at to whine went and like to sell to could sore she went i

lay with a cart-horse, and expwansing to
my tave,' I heard any wat
preteites                          evthat you did
 you.

You oldiffcon said.
 re Talk what think, --' The ormify, things, and the earther.'

The      and planbting irgred to gave no setrtied sat think that Myand at planc was to tay tingevendy down, he netting stiokly of the court insters ancurp seemed by the skiar.'

'to froin
the whole talk hing
avcance,
it wa
72183/72183 [==============================] - 73s 1ms/step - loss: 1.5360
Epoch 6/160
72128/72183 [============================>.] - ETA: 0s - loss: 1.5201
----- Generating text after Epoch: 5
----- diversity: 0.2
----- Generating with seed: "

'Not a bit,' said the Caterp"


'Not a bit,' said the Caterpillar as she had story said to herself to see the thing as she had stan in the sear that said to the thing at the this such a low to the sear streen the this some of the thing and she had the this said to the some of the this such a little to see the shool as she

ters!' shouted the Gryphon, with the soldiers of the treathers, when the courh, and the 'she not offended would be a moral the sidese as she must made--on
                                                                                                            on an the stance both doesn't go to thing was the never said, and all the tried the teards was and bole the Dormousered as she was no the mark.

'I don't must it was t
----- diversity: 1.0
----- Generating with seed: "ters!' shouted the Gryphon, wi"
ters!' shouted the Gryphon, with this like learne,          I remame off. Alice--'

' heeveime to chin sigh,' the I srin fimper, usmongling to eye,' said Alice.

'I oheK
there could, huppen!' Alice tueno with
one of the kin and scry down sifces.a
'
And he remark. There see have ma,
'the neere anything said, Sopit the poclarit?'

'the Mouse fourselt,ose you this
gome doeging her was really door for the trial such order, and
thi
72183/72183 [==============================] - 75s 1ms/s

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


here's noA'
het is w
72183/72183 [==============================] - 73s 1ms/step - loss: 1.4394
Epoch 13/160
72160/72183 [============================>.] - ETA: 0s - loss: 1.4311
----- Generating text after Epoch: 12
----- diversity: 0.2
----- Generating with seed: "child again, singing a sort of"
child again, singing a sort of the shall of the sean and the twing, and then the twing the san she was not the baby with the san she went on the suppook with the shall of the tail the stang was as they see a little thing the time and was a little sides white with the door, and they must be things and the twing the tried that they were the little three going the same the san in the time the twing the twing and there was there w
----- diversity: 0.5
----- Generating with seed: "child again, singing a sort of"
child again, singing a sort of the tall as they thought the day.

'Then I tursed your his mous,' the Mock Turtle, 'I should not minutes the mouse,' s has he a little supp, and she is of re

...

 such a rule at processions; 'she shall be heard the s one, sam the s 'she say the could while soone, 'I's,' the s oond the s wall the tet at murkigly while as in stentent trey to the s at down griden she be as soontening on with I hallse sharree, and walled heres of the looked of the kneened at
the King;
wily in somely it shares her, the lo the littlided of the dry she see and the could of brrive the wow's be hear tones of he
----- diversity: 1.0
----- Generating with seed: " such a rule at processions; '"
 such a rule at processions; 'I'' lat say the tews out of, set extrirese
again 'it's mather of
mpeasetigy in quite after tay her, sees it'tres you put of them door, and youm, fupt the be dane plate in mord 
'Wells.

'sor and the s for' coukning it
shan oy
were,' she wasVUwn for'
Alice did not the
meaner, he soonn.

Then Thes I the flor the loop sily her crose see
ses see, 'sor Alice appeaqyeeing from near eyateft toise wioond,
72183/72183 [==============================] - 73s 1ms/s

appear, and after ypneg out the omt wonle the th* to opridg iatoy, and hAntenta*pin'lylaT wgas wats the knsoownut notitat oidbgrer, wit an  to onc( the was vd po kry an o soR 'alAte   the she 8'''' thEe  and hA(ie sohlr
go w the ste: the q
pncen'ren plot reole to-dedo*sou seal
''I twontw''ding to! and '
the she at5e coneftrsenly, whi('
the she , be1'* Zf the kindion' ceq w, arenather oing ws ore co sams5 to sIRU tg
----- diversity: 0.5
----- Generating with seed: " it did not
appear, and after "
 it did not
appear, and after ypneg hj* octer his litted!'un ous beficgm, sroh uouy whery to otu--andu rorcen the not t o the theuhfp ir, samsgertyantuchd ap8ts the ktous h(kesd or theg(d[n bi thmrojn'get
hm be,norde out her thehked urs5 sag the se s itaituy the kpeaye w the to urryXt kiitR and the ang alHminle gaton! het and hA(r(m-oyt ror 'IouILU to off the ondr 
'the she nomOt whice the  'an w,  and He the beile crear,y' th
----- diversity: 1.0
----- Generating with seed: " it did not
appear

had, of uulle sad Adat Pheth haledo thale yindly youh youinse heole, the soir izmindnp h to
thinh yemsttnH anc' 'ams orteas tagek and uad tgH
Alicg siaVR nnea Hu Mas n ''hoowene  n 'Watd ani '
tothe ano  hom 'I de w lane the Albe'
whech to, the s oo
'that oea beerseowtipnoamint
72183/72183 [==============================] - 73s 1ms/step - loss: 5.7320
Epoch 88/160
72160/72183 [============================>.] - ETA: 0s - loss: 5.8079
----- Generating text after Epoch: 87
----- diversity: 0.2
----- Generating with seed: "d! Off--'

'Nonsense!' said Al"
d! Off--'

'Nonsense!' said Alaunlem bouit anooment, 'awa't the thewe'rogs,rehin  ano  ani heer th kiat 'Iu the sue q hewhlePdRery theaT to 
aninatht to a suaPoc a t oon  the the the th,' the letl It theyle 'ple an Yat then 
I ti ani
the idoilHAM, 
aninoe the thea
thereS ano,ll  aniut  ani the Al s  ben  the APtce Rte the sinienpe anl 'the AUS 'Aliht an  an o's shupe beed to there said ile no t ohu ani then wupe thed'
w_'te an
----- divers

 still it was very uncomfortabe towin y thebeHouin esblerseen t theB on ofris a tgeb* i leAicn on tonouad s noin oleb gaico s angel  once ali buese fouye6 theI toe we ththo  be, whgumeeuset pis weS dK oo aned toI' harin  le hi' s mouY,y an  h s s de tooucUr, itt the an 'anog anoti ms AAl 'ane' said.
the a Hure earo uned.

'can 'Alin oe said tie anoge a *
is ani things nolyobk ts thHIthermedjfrho houg at ants s lon 
An.
I sot t
----- diversity: 1.0
----- Generating with seed: " still it was very uncomfortab"
 still it was very uncomfortabo the 'n besd D pes Iro le fot saik oeoind whi uyeye seahe iif kpem hin h haeat veaitgtee itopf elica Dinegid  ol 
theI diwu-e paw.

isiy nesla uto halo werh m  vfen  bicet choop theris, o sofrey -

mon
lare iat ow
Adi)lead otiwtilin ri as]kuls ofoind mei(t, the ar1o
an obe th Tee to elon huin-m5s onpfe* df futon was Houane thet t rete I an  is
hers onnildte e
itd weresrgulroingumiat sreer bhe bin
72183/72183 [==============================] - 73s 1ms/s

At last the Mouse, who seemedIyn,y thedh  thers, tgu the tge the sr an   than  thek Wnor' the Anor the  ta sD an the sw' then th in f '
'the aturX hand 'the T and the san' ther ane  hanthteS  hou s thenush atyont heWt thedrWt the y 'sant the sa wases the'rle theg theCsb  te then 'the ye 'as to on t the be the sa' sot tes anhte ur Tho se whe he hon G the y thed hery gog  the w mhoU t the y  thed
ths hinyoHn  sane  'the sr tho
----- diversity: 0.5
----- Generating with seed: "

At last the Mouse, who seeme"


At last the Mouse, who seeme s the e Wof the w
Whou he s sone andhaoadr thes  thirfv al v hhrushoctPin s hanIdhe heqmar1s tha Innun Wunoulti ad8t noUe henT saidne susa in  in  chohsaiditr sin  andes theeIre yon hhatHAe semse tgoir6 anduin alot 

and an whie  as thou  we y an the yorerof as sheuto ra thecI-ther  son itse 'shing t sh ' tf  Alicer.' 'the
atoth  salec Wheb Git hee
sTe  foory, Alic he Ale sanin et amekle th boker
----- diversity: 1.0
----- Generating with seed: "

At las